code start here

In [0]:
import requests
import cv2
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
API_KEY = "4248579e1322494d8283ea682649f437"
PATH = '/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/training_set/'
MAX_RESULTS = 250
GROUP_SIZE = 50

In [0]:

if not os.path.isdir(PATH):
    os.mkdir(PATH)

# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"


In [0]:
pokemons = []

while True:
    name = input("Enter pokemon Name: ").lower()
    if not name:
        break
    pokemons.append(name)


In [0]:
pokemons

Bulbasaur, Charmander and Pikachu.

In [0]:

for term in pokemons:
    # headers and search parameters
    headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
    params = {"q": term, "offset": 0, "count": GROUP_SIZE}
    
    # make the search
    print("[INFO] searching Bing API for '{}'".format(term))
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    
    # grab the results from the search, including the total number of
    # estimated results returned by the Bing API
    results = search.json()
    estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
    print("[INFO] {} total results for '{}'".format(estNumResults,
    	term))
     
    # initialize the total number of images downloaded thus far
    total = 0
    folder = PATH + term

    if not os.path.isdir(folder):
        os.mkdir(folder)
        
    # loop over the estimated number of results in `GROUP_SIZE` groups
    for offset in range(0, estNumResults, GROUP_SIZE):
        # update the search parameters using the current offset, then
        # make the request to fetch the results
        print("[INFO] making request for group {}-{} of {}...".format(offset, offset + GROUP_SIZE, estNumResults))
        params["offset"] = offset
        search = requests.get(URL, headers=headers, params=params)
        search.raise_for_status()
        results = search.json()
        print("[INFO] saving images for group {}-{} of {}...".format(offset, offset + GROUP_SIZE, estNumResults))
        
        # loop over the results
        for v in results["value"]:
            # try to download the image
            try:
                # make a request to download the image
                print("[INFO] fetching: {}".format(v["contentUrl"]))            
                r = requests.get(v["contentUrl"], timeout=30)
                 
                # build the path to the output image
                ext = v["contentUrl"][v["contentUrl"].rfind("."):]
                img_type = v["encodingFormat"]
                
                p = os.path.sep.join([folder, "{}.{}".format(str(total).zfill(6), img_type)])
                
                
                if str(img_type) != "animatedgif":
                # write the image to disk
                    f = open(p, "wb")
                    f.write(r.content)
                    f.close()
                    total += 1
                    
                    # try to load the image from disk
                    image = cv2.imread(p)
        
                    # if the image is `None` then we could not properly load the
                    # image from disk (so it should be ignored)
                    if image is None:
                            print("[INFO-DEL] deleting: {}".format(p))
                            os.remove(p)
                            continue
     
            # catch any errors that would not unable us to download the
            # image
            except Exception as e:
                # check to see if our exception is in our list of
                # exceptions to check for
                print ("[ERROR]: ",str(e))

            
    print ("*"*50)            
    print ("[END] Images Donwloaded for",term)
    print ("*"*50)
    

[INFO] searching Bing API for 'bulbasaur'
[INFO] 250 total results for 'bulbasaur'
[INFO] making request for group 0-50 of 250...
[INFO] saving images for group 0-50 of 250...
[INFO] fetching: http://static.giantbomb.com/uploads/original/13/135472/1891758-001bulbasaur.png
[INFO] fetching: https://hawaiieesfieldsemester.files.wordpress.com/2012/04/bulbasaur.jpg
[INFO] fetching: https://theinverselook.files.wordpress.com/2015/09/1633835-bulbasaur_by_thunderwest.jpg
[INFO] fetching: http://www.toonzone.net/wp-content/uploads/2013/09/Bulbasaur_Official-Art_300dpi.jpg
[INFO] fetching: http://img2.wikia.nocookie.net/__cb20140903033456/pokemon/images/5/53/001Bulbasaur_OS_anime.png
[INFO] fetching: http://vignette3.wikia.nocookie.net/pokemon-planet/images/5/5b/Bulbasaur_by_elfaceitoso.png/revision/latest?cb=20161115042430
[INFO] fetching: http://pre05.deviantart.net/c63e/th/pre/i/2012/181/3/8/pokemon__001__bulbasaur_by_theotherdellfan-d55g23w.png
[INFO] fetching: http://th07.deviantart.net/fs7

In [0]:
p

'/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/training_setpikachu/000236.jpeg'

model start here







In [0]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


W0714 05:53:00.430471 140397552752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 05:53:00.487443 140397552752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 05:53:00.497090 140397552752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 05:53:00.549280 140397552752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0714 05:53:00.611577 140397552752512 deprecation_wrapp

In [0]:

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/training_set/',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/FSDP2019_DEEP_LEARNING/day30andcodechallenge/dataset/test_set/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')



classifier.fit_generator(training_set,
                         steps_per_epoch = 50,
                         epochs = 10,#25
                         validation_data = test_set,
                         validation_steps = 200)


Found 710 images belonging to 3 classes.
Found 16 images belonging to 3 classes.


W0714 05:53:09.845261 140397552752512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0714 05:53:09.918772 140397552752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
12/50 [======>.......................] - ETA: 1:14 - loss: 0.9462 - acc: 0.5000

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


50/50 [==============================] - 252s 5s/step - loss: 0.6513 - acc: 0.7054 - val_loss: 0.8966 - val_acc: 0.8125
Epoch 2/10
50/50 [==============================] - 238s 5s/step - loss: 0.3696 - acc: 0.8515 - val_loss: 0.9528 - val_acc: 0.8125
Epoch 3/10
50/50 [==============================] - 235s 5s/step - loss: 0.2981 - acc: 0.8985 - val_loss: 0.6166 - val_acc: 0.8125
Epoch 4/10
50/50 [==============================] - 239s 5s/step - loss: 0.2656 - acc: 0.9068 - val_loss: 0.7261 - val_acc: 0.8125
Epoch 5/10
50/50 [==============================] - 239s 5s/step - loss: 0.2263 - acc: 0.9193 - val_loss: 0.8176 - val_acc: 0.7500
Epoch 6/10
50/50 [==============================] - 234s 5s/step - loss: 0.1883 - acc: 0.9343 - val_loss: 0.6155 - val_acc: 0.8125
Epoch 7/10
50/50 [==============================] - 238s 5s/step - loss: 0.1969 - acc: 0.9336 - val_loss: 0.7435 - val_acc: 0.7500
Epoch 8/10
50/50 [==============================] - 236s 5s/step - loss: 0.1660 - acc: 0.9393 

In [4]:

# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/dataset/test_set/pikachu/000235.jpeg', target_size = (64, 64))

#test_image = image.load_img('/content/drive/My Drive/dataset/single_prediction/000233.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

# {'bulbasaur': 0, 'charmander': 1, 'pikachu': 2}

if int(result[0][0]) == 1:
    prediction = 'Bulbasaur'
elif int(result[0][1]) == 1:
    prediction = 'Charmander'
else:
    prediction = 'Pikachu'

print (prediction)

FileNotFoundError: ignored